# Lifecycle of Write Query

## Creating a Table

In [6]:
create_table = """
CREATE TABLE default.db.orders (
    order_id BIGINT,
    customer_id BIGINT,
    order_amount DECIMAL(10,2),
    order_ts TIMESTAMP
)
USING iceberg
PARTITIONED BY (HOUR(order_ts))
"""

spark.sql(create_table)

DataFrame[]

### Table Metadata
Inspecting the metadata of this newly created table in Minio, you'll be able to figure out the following:

 - Table identification
 - Schema definition
 - Partitioning
 - Table properties
 - Snapshot tracking
 - Timestamps
    
**Metadata file location:** warehouse/default/db/orders/metadata/v1.metadata.json ([File Contents](refs/metadata.json))

### Useful Queries to Inspect Iceberg Data Lakehouse

```sql
SHOW NAMESPACES IN default;
SHOW TABLES IN default.db;
SELECT * FROM default.db.orders.manifests;
SELECT * FROM default.db.orders.snapshots;
SELECT * FROM default.db.orders.files
```

If you run any of this queries, you'll find that none of them have any records in it at table creation. 

**Namespaces** serve as a logical container or grouping mechanism for tables, similar to schemas in traditional databases. Namespaces help organize tables hierarchically and provide isolation between different data domains.